# 5. Capstone Project: Data Preparation II

***

![headerall](./images/headers/header_all.jpg)

##  Goals

### Project:
In this work, we will first analyze where and when traffic congestion is highest and lowest in New York State. We will then build different machine learning models capable of predicting cab travel times in and around New York City using only variables that can be easily obtained from a smartphone app or a website. We will then compare their performance and explore the possibility of using additional variables such as weather forecasts and holidays to improve the predictive performance of the models.

### Section:
In this section, data will be prepared for the next steps, i.e., exploratory data analysis and machine learning model building. First, we will collect and import the necessary datasets. Next, we will review each of these datasets to fully understand the data and its structure, including the different variables available, their format, and their relevance to the project. Finally, we will perform data cleaning and manipulation, as well as feature engineering, and encoding.

The data preparation will be divided into two subsections, the first dedicated to the external datasets (Data Preparation Part I) and the second to the New York City Taxi Trip data set (Data Preparation Part II). Finally, external datasets will be merged with the primary one.

## Data
### External Datasets:
- Weather Forecast: The 2018 NYC weather forecast was collected from the [National Weather Service Forecast Office](https://w2.weather.gov/climate/index.php?wfo=okx) website. Daily measurements were taken from January to December 2018 in Central Park. These measures are given in imperial units and include daily minimum and maximum temperatures, precipitations, snowfall, and snow depth.

- Holidays: The 2018 NYC holidays list was collected from the [Office Holiday](https://www.officeholidays.com/countries/usa/new-york/2021) website. The dataset contains the name, date, and type of holidays for New York.

- Taxi Zones: The NYC Taxi Zones dataset was collected from the [NYC Open Data](https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc) website. It contains the pickup and drop-off zones (Location IDs) for the Yellow, Green, and FHV Trip Records. The taxi zones are based on the NYC Department of City Planning’s Neighborhood.

### Primary Datasets:

- Taxi Trips: The 2018 NYC Taxi Trip dataset was collected from the [Google Big Query](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-tlc-trips?project=jovial-monument-300209&folder=&organizationId=) platform. The dataset contains more than 100'000'000 Yellow Taxi Trip records for 2018 and contains an extensive amount of variables including the pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts.

***
## Table of Content:
    1. Data Preparation
        1.1 External Datasets
            1.1.1 Weather Forecast Dataset
            1.1.2 Holidays Dataset
            1.1.3 Taxi Zones Dataset
        1.2 Primary Dataset
            1.2.1 Taxi Trips Dataset
    2. Exploratory Data Analysis
        2.1 Primary Dataset
            2.1.1 Temporal Analysis
            2.1.2 Spatio-Temporal Analysis
        2.2 External Datasets
            2.2.1 Temporal Analysis of Weather Data
            2.2.2 Temporal Analysis of Holidays Data
        2.3 Combined Dataset
            2.3.1 Overall Features Correlation
    3. Machine Learning Models

***
## Python Libraries and Magic commands Import

In [1]:
# Import python core libraries
import os

# Import data processing libraries
import pandas as pd
import numpy as np

# Import Visualization librairies
import matplotlib.pyplot as plt
import seaborn as sns

# Import machine learning libraries
from sklearn.model_selection import train_test_split

In [2]:
# Set up magic commands
%matplotlib inline
%config Completer.use_jedi = False

***
## Data Import

In [3]:
# Import the weather forecasts dataset
weather_df = pd.read_pickle(r'data/processed/weather.pickle')

# Import the holidays dataset
holidays_df = pd.read_pickle(r'data/processed/holidays.pickle')

# Import the zones dataset
zones_df = pd.read_pickle(r'data/processed/zones.pickle')

***
## Functions import

In [4]:
# Define a function that transform column headers to titles
def to_title(data, sep='_'):
    return [s.replace(sep, ' ').title() for s in data]

In [5]:
# Define a function that compute and display descriptive statistics for selected columns
def outliers_distribution(data, columns, statistics, percentiles):

    # Create an empty dictionnary to collect the columns' descriptive statistics
    frame = {}

    # Iterate over the given numerical columns
    for column in columns:

        # Compute the descriptive statistics for records above the column's given percentile
        output = (
            data.loc[data[column] > percentiles[column].iloc[0], column]
            .describe()[statistics]
            .round(2)
        )

        # Add the column's descriptive statistics to the dictionnary
        frame[column] = output

    # Return the columns' descriptive statistics as a data frame
    return pd.DataFrame(frame)

In [6]:
# Define a function that display scatter plots of selected numerical columns
def plot_scatter(data, columns, colnames, variable, ncols=1, nrows=1):

    varname = variable.replace("_", " ").title()

    # Create a figure with n columns and rows
    fig, axes = plt.subplots(
        nrows=nrows, ncols=ncols, sharey=True, figsize=(ncols * 5, nrows * 5)
    )

    # Flatten axes for 2D figures
    if ncols > 1 or nrows > 1:
        axes = axes.ravel()

    # Generate the plots
    for ax, col, colname in zip(axes, columns, colnames):
        sns.scatterplot(x=col, y=variable, data=data, ax=ax, s=10)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.set_ylim(0)
        ax.tick_params(labelsize=14)
        ax.set_xlabel(colname, fontsize=14)
        ax.set_ylabel(varname, fontsize=14)
        ax.set_title(f"{varname} vs {colname}", fontsize=16)
        ax.grid()

    # Adjust the padding between and around subplots
    fig.tight_layout(pad=0.5, w_pad=0.5)

    # Display the figure
    plt.show()

In [7]:
# Define a function that display scatter plots of selected numerical columns
def plot_histplot(data, columns, colnames, ncols=1, nrows=1):

    # Create a figure with n columns and rows
    fig, axes = plt.subplots(
        nrows=nrows, ncols=ncols, sharey=True, figsize=(ncols * 5, nrows * 5)
    )

    # Flatten axes for 2D figures
    if ncols > 1 or nrows > 1:
        axes = axes.ravel()

    # Generate the plots
    for ax, col, colname in zip(axes, columns, colnames):
        sns.histplot(x=col, bins=20, data=data, ax=ax, edgecolor="white", alpha=0.7)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.tick_params(labelsize=14)
        ax.set_xlabel(colname, fontsize=14)
        ax.set_ylabel("Count", fontsize=14)
        ax.set_title(colname, fontsize=16)
        ax.grid()

    # Adjust the padding between and around subplots
    fig.tight_layout(pad=0.5, w_pad=0.5)

    # Display the figure
    plt.show()

In [8]:
# Define a function that display individual boxplot of selected numerical columns
def plot_boxplot(data, columns, colnames, ncols=1, nrows=1):
    # Create a figure with n columns and rows
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 5))

    # Flatten axes for 2D figures
    if ncols > 1 or nrows > 1:
        axes = axes.ravel()

    # Generate the plots
    for ax, col, colname in zip(axes, columns, colnames):
        sns.boxplot(
            y=col, data=data, ax=ax, width=0.35, linewidth=1, boxprops=dict(alpha=0.7)
        )
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.yaxis.set_ticks_position("none")
        ax.xaxis.set_ticks([])
        ax.tick_params(labelsize=14)
        ax.set_xlabel(colname, fontsize=14)
        ax.set_ylabel(None)
        ax.set_title(colname, fontsize=16)
        ax.grid()

    # Adjust the padding between and around subplots
    fig.tight_layout(pad=0.5, w_pad=0.5)

    # Display the figure
    plt.show()

In [9]:
# Define a function that display individual violinplot of selected numerical columns
def plot_violinplot(data, columns, colnames, ncols=1, nrows=1, color="C0"):

    # Create a figure with n columns and rows
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 5))

    # Flatten axes for 2D figures
    if ncols > 1 or nrows > 1:
        axes = axes.ravel()

    # Generate the plots
    for ax, col, colname in zip(axes, columns, colnames):
        violin_plot = ax.violinplot(data[col])

        # Change color of the violin plot
        for pc in violin_plot["bodies"]:
            pc.set_facecolor(color)
            pc.set_edgecolor(color)

        violin_plot["cbars"].set_edgecolor("black")
        violin_plot["cmins"].set_edgecolor("black")
        violin_plot["cmaxes"].set_edgecolor("black")

        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.yaxis.set_ticks_position("none")
        ax.tick_params(labelsize=14)
        ax.set_title(colname, fontsize=16)
        ax.grid()

    # Adjust the padding between and around subplots
    fig.tight_layout(pad=0.5, w_pad=0.5)

    # Display the figure
    plt.show()

***
# 1. Data Preparation
## 1.2 Data Preparation: NYC taxi trip dataset
### Goal:
Explore a subset (10%) of the 2018 NYC yellow taxi trip dataset dataset and perform data cleaning and manipulation, as well as feature engineering and encoding if needed.
### Code:

In [11]:
# Create a global variable containing the credentials to query Google Big Query Servers
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"credentials\exts-ml-5e5656f1d391.json"

# Create an empty data frame
records_df = pd.DataFrame()

# Create a row offest
row_offset = 0

# Create a row limit
row_limit = 10000000

# Create a row offest counter
row_counter = 10000000

# Get the NYC taxi dataset in chuncks of 10M to avoid insufficient memory issues
while row_counter == row_limit:

    print(f"Fetching rows from {row_offset} to {row_offset + row_counter}:")

    # Create a query to fetch and transform the NYC taxi dataset from the Google Big Query servers
    query = f"""
     SELECT
         pickup_datetime,
         dropoff_datetime,
         passenger_count,
         trip_distance,
         tolls_amount,
         fare_amount,
         pickup_location_id,
         dropoff_location_id,
         TIMESTAMP_DIFF(TIMESTAMP(pickup_datetime), TIMESTAMP(dropoff_datetime), MINUTE) as trip_duration
     FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018`
     LIMIT {row_limit}
     OFFSET {row_offset};
     """
    # Get a chunck of the Google Big Query servers
    df = pd.read_gbq(
        query, project_id="exts-ml", dialect="standard", progress_bar_type="tqdm"
    )

    # Get a random sample of 1% of the NYC taxi dataset chunck
    records_df = records_df.append(
        df.sample(frac=0.010, replace=False, random_state=0), ignore_index=True
    )

    # Update the row counter with the current data frame lenght
    row_counter = len(df.index)

    # Update the row offest
    row_offset += row_counter

Fetching rows from 0 to 10000000:


Downloading: 100%|██████████████| 10000000/10000000 [25:50<00:00, 6451.24rows/s]


Fetching rows from 10000000 to 20000000:


Downloading: 100%|██████████████| 10000000/10000000 [24:53<00:00, 6696.77rows/s]


Fetching rows from 20000000 to 30000000:


...

Fetching rows from 70000000 to 80000000:


Downloading: 100%|██████████████| 10000000/10000000 [23:38<00:00, 7051.84rows/s]


Fetching rows from 80000000 to 90000000:


Downloading:  35%|█████▎         | 3517983/10000000 [38:09<14:21, 7526.53rows/s]

GenericGBQException: Reason: Deadline of 600.0s exceeded while calling functools.partial(functools.partial(<bound method JSONConnection.api_request of <google.cloud.bigquery._http.Connection object at 0x10cd61880>>, timeout=300.0, method='GET', path='/projects/exts-ml/datasets/_888f04df2bf18df6709faaa7f2ca1793859c041d/tables/anonf5e544d497bfcceded2fcf1e334537a57e13d71c/data', query_params={'pageToken': 'BEOATIW6PMAQAAASAUIIBAEAAUNAWCE73TLACEH777776BZA77777777777767ZKBEEAAEADDDTJJNYB', 'formatOptions.useInt64Timestamp': True}, headers={'X-Server-Timeout': '300.0', 'Accept-Encoding': 'gzip', 'X-Goog-API-Client': 'pandas-1.3.1 gl-python/3.9.6 grpc/1.38.1 gax/1.22.4 gapic/2.26.0 gccl/2.26.0', 'User-Agent': 'pandas-1.3.1 gl-python/3.9.6 grpc/1.38.1 gax/1.22.4 gapic/2.26.0 gccl/2.26.0'})), last exception: HTTPSConnectionPool(host='bigquery.googleapis.com', port=443): Read timed out.

In [ ]:
# Open data frame
#records_df = pd.read_pickle('data/raw/taxi_records_v1.pickle')

#records_df['trip_duration'] = (
#   records_df.dropoff_datetime - records_df.pickup_datetime
#).dt.seconds / 60

#records_df['pickup_location_id'] = records_df['pickup_location_id'].astype('int64')
#records_df['dropoff_location_id'] = records_df['dropoff_location_id'].astype('int64')

In [ ]:
# Display the first five rows of the data frame
records_df.head()

In [ ]:
# Display a quick description of the data frame
records_df.info()

**Notes:** the data frame contains 1% of all 2018 NYC cab fare records, totaling 11,223,463 data points. Most of the numeric columns are the wrong type and will need to be converted from string (Object) to float or integer. Columns may also contain null values, and since most machine learning algorithms cannot properly process them, we will first need to check that each column does not contain any before continuing with data cleaning and feature engineering/coding.

In [ ]:
# Display the number of empty cells in each column of the data frame
records_df.isna().sum()

**Notes:** the dataset does not contain any null values.

In [ ]:
# Define numerical columns to be converted to float
num_cols = ['trip_distance', 'tolls_amount', 'fare_amount', 'trip_duration']

# Convert numerical columns to float
records_df[num_cols] = records_df[num_cols].astype('float64')

# Display a quick description of the data frame
records_df.info()

**Notes:** the columns have been correctly converted to float. Values of numeric columns will be converted to metric units. In addition, columns containing dates will be checked for records dating from before or after 2018.

In [ ]:
# Convert the trip distance from miles to kilometers
records_df["trip_distance"] = np.multiply(records_df["trip_distance"], 1.60934)

# Display the records with pickup dates out of the date range (2018.01.01 - 2018.01.01)
records_df[
    (records_df['pickup_datetime'] < '2018.01.01')
    | (records_df['pickup_datetime'] > '2019.01.01')
]

**Notes:** over a dozen records date from before or after 2018. These records will be removed from the dataset.

In [ ]:
# Remove records with pickup dates below 2018.01.01 and above 2018.01.01
records_df = records_df[
    (records_df['pickup_datetime'] >= '2018.01.01')
    & (records_df['pickup_datetime'] <= '2019.01.01')
]

# Display a quick description of the data frame
records_df.info()

**Notes:** the 16 records with dates out of range were removed from the dataset.

In [ ]:
# Display some descriptive statistics
records_df.describe().round(2)

**Notes:** Most numerical columns contain extreme and erroneous values. For instance, according to the [official NYC Taxi](https://www1.nyc.gov/site/tlc/passengers/passenger-frequently-asked-questions.page) website, the maximum number of passengers allowed by law in a yellow taxi is four in a four-passenger taxi or five in a five-passenger taxi. Therefore, trips with more than five passengers are either errors or the result of taxi drivers picking up additional passengers en route to their final destination. Some records have negative, null or extreme values the number of passenger, distance traveled, toll amount and fare amount. Taxi trip records with null, extreme or erroneous values will be removed from the dataset.

In [ ]:
# Define the name of numerical columns
num_cols = [
    'passenger_count',
    'trip_distance',
    'tolls_amount',
    'fare_amount',
    'trip_duration',
]

# Define the name of numerical columns
num_colnames = to_title(num_cols)

In [ ]:
# Display scatterplot of numerical columns
plot_scatter(records_df, num_cols, num_colnames, 'trip_duration', ncols=len(num_cols))

In [ ]:
# Display histograms of numerical columns
plot_histplot(records_df, num_cols, num_colnames, ncols=len(num_cols))

In [ ]:
# Display boxplots of numerical columns
plot_boxplot(records_df, num_cols, num_colnames, ncols=len(num_cols))

**Notes:** The distribution of variables such as number of passengers, distance traveled, toll amount, trip amount, and trip duration is heavily skewed by the presence of many near-zero values and outliers. We will not eliminate the near-zero values because they correspond to real and frequent trips, but we will eliminate most of the outliers while trying to preserve some degree of variability so that our models can correctly estimate the duration of non-conventional taxi trips.

In [ ]:
# Compute the numerical columns' 75, 95, and 99th percentiles
percentiles_df = records_df.quantile([0.75, 0.95, 0.99])[num_cols].reset_index()

# Display the numerical columns' 75, 95, and 99th percentiles
percentiles_df

**Notes:** the table above shows the 75th, 95th, and 99th of numerical variables, including passagenger count, trip distance, tolls and fare amounts, and trip duration.

In [ ]:
# Define the descriptive statistics to display
perc_statistics = ['count', 'mean', 'std', 'min', 'max']

# Get the numerical columns' 75th percentiles
perc_75 = percentiles_df[percentiles_df['index'] == 0.75]

# Compute and display descriptive statistics for records above the column's 75th percentile
outliers_distribution(records_df, num_cols, perc_statistics, perc_75)

**Notes:** the table above shows the descriptive statistics of the numerical variables for records marked as outliers, if the 75th percentile was used as the upper threshold. Depending on the variable, up to one-third of the data set could be didscarded and only the most common taxi trips would be retained.

In [ ]:
# Get the numerical columns' 95th percentiles
perc_95 = percentiles_df[percentiles_df['index'] == 0.95]

# Compute and display descriptive statistics for records above the column's 95th percentile
outliers_distribution(records_df, num_cols, perc_statistics, perc_95)

**Notes:** the table above shows the descriptive statistics of the numerical variables for records marked as outliers, if the 95th percentile was used as the upper threshold. Depending on the variable, up to 7% of the data set could be discarded, but trips diversity would be greater than if the 75th percentile were used. It would also discard taxi trips with more than five passengers. However, for the other variables, using the 95th percentile would lead to the exclusion of long-distance and flat-rate rides, including those between Manhattan and JFK Airport, as reported in the [official NYC FAQ](https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page).

In [ ]:
# Get the numerical columns' 99th percentiles
perc_99 = percentiles_df[percentiles_df['index'] == 0.99]

# Compute and display descriptive statistics for records above the column's 99th percentile
outliers_distribution(records_df, num_cols, perc_statistics, perc_99)

**Notes:** the table above shows the descriptive statistics of the numerical variables for records marked as outliers, if the 99th percentile was used as the upper threshold.  Depending on the variable, up to 1.5% of the data set could be discarded. The diversity of trips would be much greater than if the 75th and 95th percentiles were used. By using the 99th percentile, the most extreme or erroneous trips would be discarded, while retaining a high degree of variability, which would help our predictive model estimate the duration of non-conventional cab trips. We will thus use mostly the 99th and 95th percentil to delete outliers from the data set.

In [ ]:
# Discards records with passenger counts above the 95th percentile and below or equal to 0
records_df = records_df.loc[
    (records_df['passenger_count'] <= percentiles_df['passenger_count'].iloc[1])
    & (records_df['passenger_count'] > 0)
]

# Discards records with toll amounts above the 99th percentile and below or equal to 0
records_df = records_df.loc[
    (records_df['trip_distance'] <= percentiles_df['trip_distance'].iloc[2])
    & (records_df['trip_distance'] > 0)
]

# Discards records with tolls amounts above the 99th percentile and below 0
records_df = records_df.loc[
    (records_df['tolls_amount'] <= percentiles_df['tolls_amount'].iloc[2])
    & (records_df['trip_distance'] >= 0)
]

# Discards records with fare amounts above the 99th percentile and below 0
records_df = records_df.loc[
    (records_df['fare_amount'] <= percentiles_df['fare_amount'].iloc[2])
    & (records_df['fare_amount'] > 0)
]

# Discards records with trip duration above the 99 percentile and below 1
records_df = records_df.loc[
    (records_df['trip_duration'] <= percentiles_df['trip_duration'].iloc[2])
    & (records_df['trip_duration'] > 1)
]

# Display some descriptive statistics
records_df.describe().round(2)

**Note:** The 95th percentile was used to discard outliers from the passenger count column, while the 99th percentile was used to discard outliers and erroneous records from all other columns. Approximately 7% of the data set was discarded.

In [ ]:
# Display scatterplots of numerical columns
plot_scatter(records_df, num_cols, num_colnames, 'trip_duration', ncols=len(num_cols))

In [ ]:
# Display histograms of numerical columns
plot_histplot(records_df, num_cols, num_colnames, ncols=len(num_cols))

In [ ]:
# Display boxplots of numerical columns
plot_boxplot(records_df, num_cols, num_colnames, ncols=len(num_cols))

**Notes:** After removing the extreme and spurious values from the data set, we can observe a strong correlation between travel time and distance and fare amount. In addition, each variable has a righ-skewed distribution. Therefore, we will need to normalize the data before using it to train our different models. Below, we will proceed with feature engineering and then encoding, starting with the date and time of pickup.

In [ ]:
# Add a new column containing the date
records_df["pickup_date"] = records_df["pickup_datetime"].dt.date

# Add a new column containing the month of the year
records_df["pickup_month"] = records_df["pickup_datetime"].dt.month

# Add a new column containing the week of the year
records_df["pickup_week"] = records_df["pickup_datetime"].dt.isocalendar().week

# Add a new column containing the day of the year
records_df["pickup_yearday"] = records_df["pickup_datetime"].dt.dayofyear

# Add a new column containing the weekday of the week
records_df["pickup_weekday"] = records_df["pickup_datetime"].dt.weekday

# Add a new column containing the type of weekday (week=0, weekend=1)
records_df["pickup_weekday_type"] = np.where(records_df["pickup_weekday"] < 5, 0, 1)

# Add a new column containing the hour of the day
records_df["pickup_hour"] = records_df["pickup_datetime"].dt.hour

# Add a new column containing the type of hour (night=0, day=1)
records_df["pickup_hour_type"] = np.where(
    (records_df["pickup_hour"] < 6) | (records_df["pickup_hour"] > 21), 0, 1
)

# Display the first five rows of the data frame
records_df.head()

**Notes:** using on the pickup date and time column, we created seven new variables, including the date, month of year, day of year, day of week, type of day (weekday, weekend), time of day, and peak hours. The time range used for the peak hours was determined using the daily traffic data analysis available on the [tomtom website](https://www.tomtom.com/en_gb/traffic-index/new-york-traffic/).

In [ ]:
# Display a quick description of the data frame
records_df.info()

**Notes:** with the exception of the date column, seven new features have been created. The date column will be used below to merge the weather forecast and holidays data frames with the primary data set.

In [ ]:
# Convert pickup date to datetime
records_df['pickup_date'] = pd.to_datetime(records_df['pickup_date'])

In [ ]:
# Add the weather forecast data to the main dataset
records_df = pd.merge(
    records_df,
    weather_df,
    how="left",
    left_on="pickup_date",
    right_on="date",
)

# Add the holidays data to the main dataset
records_df = pd.merge(
    records_df,
    holidays_df,
    how="left",
    left_on="pickup_date",
    right_on="holiday_date",
)

# Drop datetime columns
records_df.drop(columns=records_df.select_dtypes(include=["datetime64"]), inplace=True)

# Display a quick description of the data frame
records_df.info()

**Notes:** after merging the weather and vacation data frame with the primary data set, the NaNs in the vacation and vacation type columns will need to be filled with zeros to indicate no vacations on those days.

In [ ]:
# Fill missing values from the holidays_type with 0
records_df["holiday_type"] = records_df["holiday_type"].fillna(0)

# Fill missing values from the holidays with 0
records_df["holiday"] = records_df["holiday"].fillna(0)

# Display a quick description of the data frame
records_df.info()

**Notes:** null values have been replaced by zeros.

In [ ]:
# Add the zones data, i.e. pickup_borough_id to the main dataset
records_df = (
    pd.merge(
        records_df,
        zones_df[["LocationID", "BoroughID"]],
        how="left",
        left_on="pickup_location_id",
        right_on="LocationID",
    )
    .rename(columns={"BoroughID": "pickup_borough_id"})
    .drop("LocationID", axis="columns")
)

# Add the zones data, i.e. pickup_borough_id to the main dataset
records_df = (
    pd.merge(
        records_df,
        zones_df[["LocationID", "BoroughID"]],
        how="left",
        left_on="dropoff_location_id",
        right_on="LocationID",
    )
    .rename(columns={"BoroughID": "dropoff_borough_id"})
    .drop("LocationID", axis="columns")
)

# Display a quick description of the data frame
records_df.info()

**Notes:** the pickup and dropoff borough columns contain null values. 

In [ ]:
# Display the list of pickup location id where pickup borough was null
records_df.loc[records_df["pickup_borough_id"].isna(), "pickup_location_id"].unique()

In [ ]:
# Display the list of dropoff location id where drop borough was null
records_df.loc[records_df["dropoff_borough_id"].isna(), "dropoff_location_id"].unique()

**Notes:** These null values are due to pickup location 264 and 265, which are used as surrogates when the origin or destination of trips is unknown. we can therefore safely drop these records from the dataset.

In [ ]:
# Drop all rows containing NaNs
records_df.dropna(inplace=True)

# Display a quick description of the data frame
records_df.info()

**Notes:** records with unknown origins and destinations have been removed from the dataset. Below, we will create an additional variable, to indicate whether the trip was made in the same borough. We will also bin the toll amounts into three separate categories ranging from 0 to 3, with 0 indicating no tolls and 3 indicating the highest amount. Finaly we will drop features that should not be used by our model for the prediction of trip time duration.

In [ ]:
# Create a new column indicating if the trip was within the same borough
records_df["trip_within_borough"] = np.where(
    records_df["pickup_borough_id"] == records_df["dropoff_borough_id"], 1, 0
)

# Create a new column of binned tolls amounts
records_df["toll"] = records_df["tolls_amount"].apply(
    lambda x: 0 if x == 0 else (1 if x < 3 else 2)
)
# Drop other columns
records_df.drop(
    columns=["passenger_count", "fare_amount", "tolls_amount"], inplace=True
)

# Display a quick description of the data frame
records_df.info()

***
## Data Export

In [ ]:
# Split the dataset into a train and test data frame
train_df, test_df = train_test_split(records_df, test_size=0.2, random_state=0)

# Export the train data frame as a pickle file
train_df.to_pickle(r'data/processed/train.pickle')

# Export the test data frame as a pickle file
test_df.to_pickle(r'data/processed/test.pickle')